In [39]:
%run data_read.py

X_train, X_test, y_train, y_test = train_test_split(data['comment'], data['score_norm'], test_size=0.2, random_state=42)

Data path: C:\Users\Pop\Documents\GitHub\utcc_independent_study\training\..\data\300_data_pop.xlsx
data_size: 318
variable: data train_data test_data


1. Linear Model

In [47]:
pipeline_linear = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('regressor', LinearRegression())
])

param_distributions_linear = {
    'regressor__fit_intercept': [True, False],
    'regressor__copy_X': [True, False]
}

# Define the random search parameters
n_iter = 50  # Number of hyperparameter combinations to try
cv = 3  # Number of cross-validation folds

# Perform the random search for Linear Regression
print('Performing random search for Linear Regression...')
random_search_linear = RandomizedSearchCV(
    pipeline_linear,
    param_distributions=param_distributions_linear,
    n_iter=n_iter,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Fit the random search object on the training data
random_search_linear.fit(X_train, y_train)

# Print the best hyperparameters and corresponding mean test score
print(f'Best hyperparameters for Linear Regression: {random_search_linear.best_params_}')
print(f'Mean test score: {random_search_linear.best_score_:.4f}')

# Evaluate the best model on the test set
y_pred = random_search_linear.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'Linear Regression MSE on test set: {mse:.4f}')
print(f'Linear Regression MAE on test set: {mae:.4f}')

# Evaluate the best model on the test set and create DataFrame with results
result = pd.DataFrame({'comment': X_test,
                        'actual': y_test,
                        'predicted': random_search_linear.predict(X_test)})
result['predicted'] = result['predicted'].clip(lower=0, upper=1)  # Clip predictions to valid score range of 0 to 1

# Print the DataFrame with results
sorted_df = result.sort_values(by='predicted', ascending=False)

for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Performing random search for Linear Regression...
Fitting 3 folds for each of 4 candidates, totalling 12 fits


C:\Users\Pop\anaconda3\envs\test\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best hyperparameters for Linear Regression: {'regressor__fit_intercept': False, 'regressor__copy_X': True}
Mean test score: 0.4608
Linear Regression MSE on test set: 0.0374
Linear Regression MAE on test set: 0.1493
comment: คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความเร็วในการจัดส่งดีมากๆ การให้บริการจากร้านค้าดีมาก ถูกใจมากๆ ทุกอย่าง✨
actual: 0.8888888888888888
predicted: 1.0

comment: จัดส่งเร็วค่ะ ของได้ตรงรูปตรงปกทั้งเมาส์ทั้งแป้นพิมพ์ ส่วนตัวรู้สึกว่าแป้นพิมพ์บางไปหน่อยแต่ก็สมกับราคานี้ค่ะ มาที่เมาส์แนะนำว่าให้หาแผ่นรองเมาส์ด้วยนะคะสำหรับใครที่ไม่มี เพราะถ้าใช้กับโต๊ะเปล่าๆเลเซอร์จะไปติดค่ะ โดยรวมถือว่าดีคุ้มค่าทั้งราคาและระยะเวลาการส่งของค่ะ😊
actual: 0.7777777777777778
predicted: 1.0

comment: จัดส่งไวมากค่ะ ราคาก็ถูกดี เชื่อมต่อง่ายมาก เวลาใช้ค่อนข้างลื่น ไวค่ะ ก็ถนัดดี ไว้จะอุดหนุนใหม่นะคะ
actual: 0.7777777777777778
predicted: 1.0

comment: ก่อนซื้อก็อุตส่าห์ดูในรีวิวแล้วนะ ก็เห็นว่าเป็นปุ่มไทยแท้ แต่พอได้รับสินค้ามาก็รู้สึกผิดหวังมาก เป็นสติ๊กเกอร์ไทยซะงั้น ซึ่งพอติดสติ๊กเกอร์แล

2. Support Vector Machine

In [40]:
# Define the pipeline and parameter search space for SVR
pipeline_svm = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('regressor', SVR())
])

param_grid_svm = {
    'regressor__kernel': ['linear', 'poly', 'rbf'],
    'regressor__C': [0.1, 1, 10],
    'regressor__gamma': ['scale', 'auto'],
}

# Perform the grid search for SVR
print('Performing grid search for SVR...')
grid_search_svm = GridSearchCV(
    pipeline_svm,
    param_grid=param_grid_svm,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit the grid search object on the training data
grid_search_svm.fit(X_train, y_train)

# Print the best hyperparameters and corresponding mean test score
print(f'Best hyperparameters for SVR: {grid_search_svm.best_params_}')
print(f'Mean test score: {grid_search_svm.best_score_:.4f}')

# Evaluate the best model on the test set
y_pred = grid_search_svm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'SVR MSE on test set: {mse:.4f}')
print(f'SVR MAE on test set: {mae:.4f}')

# Evaluate the best model on the test set and create DataFrame with results
result = pd.DataFrame({'comment': X_test,
                        'actual': y_test,
                        'predicted': grid_search_svm.predict(X_test)})
result['predicted'] = result['predicted'].clip(lower=0, upper=1)  # Clip predictions to valid score range of 0 to 1

# Print the sorted DataFrame with results
sorted_df = result.sort_values(by='predicted', ascending=False)
for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Performing grid search for SVR...
Fitting 3 folds for each of 18 candidates, totalling 54 fits


C:\Users\Pop\anaconda3\envs\test\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best hyperparameters for SVR: {'regressor__C': 1, 'regressor__gamma': 'scale', 'regressor__kernel': 'linear'}
Mean test score: 0.4665
SVR MSE on test set: 0.0289
SVR MAE on test set: 0.1356
comment: คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความเร็วในการจัดส่งดีมากๆ การให้บริการจากร้านค้าดีมาก ถูกใจมากๆ ทุกอย่าง✨
actual: 0.8888888888888888
predicted: 0.9248276953853327

comment: ได้รับของครบถ้วนค่ะ การจัดส่งไว สินค้าดูดี ราคาไม่แพง คุ้มค่ามากๆ ครั้งต่อไปสั่งอีกแน่นอนค่ะ สายสวยๆ น่ารักๆ
actual: 0.7777777777777778
predicted: 0.9244675344221502

comment: ก่อนซื้อก็อุตส่าห์ดูในรีวิวแล้วนะ ก็เห็นว่าเป็นปุ่มไทยแท้ แต่พอได้รับสินค้ามาก็รู้สึกผิดหวังมาก เป็นสติ๊กเกอร์ไทยซะงั้น ซึ่งพอติดสติ๊กเกอร์แล้ว มันมองไม่เห็นตัวอักษรเลย เวลาใช้งานกลางคืน ลำบากและหงุดหงิดมาก ซื้อแบบมีไฟ แต่ก็เหมือนไม่มีไฟ ช่างเป็นไฟที่ไร้ค่าเสียจริง ผิดหวังจริงๆ รู้งี้ ซื้อร้านอื่น ที่เป็นแป้นไทยแท้ดีกว่า
actual: 0.8888888888888888
predicted: 0.8855036882063271

comment: ได้รับสินค้าเร็วมากภายในไม่กี่วัน เนื่องจากส่งจากในไทยเล

In [41]:
pipeline_tree = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('regressor', DecisionTreeRegressor())
])

param_grid_tree = {
    'regressor__max_depth': [5, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

# Perform the grid search for DecisionTreeRegressor
print('Performing grid search for DecisionTreeRegressor...')
grid_search_tree = GridSearchCV(
    pipeline_tree,
    param_grid=param_grid_tree,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit the grid search object on the training data
grid_search_tree.fit(X_train, y_train)

# Print the best hyperparameters and corresponding mean test score
print(f'Best hyperparameters for DecisionTreeRegressor: {grid_search_tree.best_params_}')
print(f'Mean test score: {grid_search_tree.best_score_:.4f}')

# Evaluate the best model on the test set
y_pred = grid_search_tree.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'DecisionTreeRegressor MSE on test set: {mse:.4f}')
print(f'DecisionTreeRegressor MAE on test set: {mae:.4f}')

# Evaluate the best model on the test set and create DataFrame with results
result = pd.DataFrame({'comment': X_test,
                        'actual': y_test,
                        'predicted': grid_search_tree.predict(X_test)})
result['predicted'] = result['predicted'].clip(lower=0, upper=1)  # Clip predictions to valid score range of 0 to 1

# Print the sorted DataFrame with results
sorted_df = result.sort_values(by='predicted', ascending=False)
for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Performing grid search for DecisionTreeRegressor...
Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\Pop\anaconda3\envs\test\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best hyperparameters for DecisionTreeRegressor: {'regressor__max_depth': 5, 'regressor__min_samples_split': 5}
Mean test score: 0.2892
DecisionTreeRegressor MSE on test set: 0.0564
DecisionTreeRegressor MAE on test set: 0.1766
comment: ตะมุตะมิน่ารัก  เสียงแจ่มดีค่ะ  ชอบมาก  ราคาไม่แพงด้วย
actual: 0.3333333333333333
predicted: 1.0

comment: ได้รับสินค้าแล้วว ส่งก่อนกำหนด4วันเลย สินค้าตรงตามปก ราคาดีไม่แพง เคยใช้แล้วชอบ รู้สึกป้องกันลิปเปื้อนแมสได้ดี
actual: 0.8888888888888888
predicted: 1.0

comment: ได้รับของครบถ้วนค่ะ การจัดส่งไว สินค้าดูดี ราคาไม่แพง คุ้มค่ามากๆ ครั้งต่อไปสั่งอีกแน่นอนค่ะ สายสวยๆ น่ารักๆ
actual: 0.7777777777777778
predicted: 0.9722222222222222

comment: คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความเร็วในการจัดส่งดีมากๆ การให้บริการจากร้านค้าดีมาก ถูกใจมากๆ ทุกอย่าง✨
actual: 0.8888888888888888
predicted: 0.9259259259259259

comment: จัดส่งเร็วค่ะ ของได้ตรงรูปตรงปกทั้งเมาส์ทั้งแป้นพิมพ์ ส่วนตัวรู้สึกว่าแป้นพิมพ์บางไปหน่อยแต่ก็สมกับราคานี้ค่ะ มาที่เมาส์แนะนำว่าให้หาแผ่นรอ

In [42]:
pipeline_knn = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer)),
    ('regressor', KNeighborsRegressor())
])

param_grid_knn = {
    'regressor__n_neighbors': [5, 10, 15],
    'regressor__weights': ['uniform', 'distance'],
    'regressor__p': [1, 2]
}

# Perform the grid search for KNeighborsRegressor
print('Performing grid search for KNeighborsRegressor...')
grid_search_knn = GridSearchCV(
    pipeline_knn,
    param_grid=param_grid_knn,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit the grid search object on the training data
grid_search_knn.fit(X_train, y_train)

# Print the best hyperparameters and corresponding mean test score
print(f'Best hyperparameters for KNeighborsRegressor: {grid_search_knn.best_params_}')
print(f'Mean test score: {grid_search_knn.best_score_:.4f}')

# Evaluate the best model on the test set
y_pred = grid_search_knn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'KNeighborsRegressor MSE on test set: {mse:.4f}')
print(f'KNeighborsRegressor MAE on test set: {mae:.4f}')

# Evaluate the best model on the test set and create DataFrame with results
result = pd.DataFrame({'comment': X_test,
                        'actual': y_test,
                        'predicted': grid_search_knn.predict(X_test)})
result['predicted'] = result['predicted'].clip(lower=0, upper=1)  # Clip predictions to valid score range of 0 to 1

# Print the sorted DataFrame with results
sorted_df = result.sort_values(by='predicted', ascending=False)
for index, row in sorted_df.iterrows():
    print(f"comment: {row['comment']}")
    print(f"actual: {row['actual']}")
    print(f"predicted: {row['predicted']}")
    print('')

Performing grid search for KNeighborsRegressor...
Fitting 3 folds for each of 12 candidates, totalling 36 fits


C:\Users\Pop\anaconda3\envs\test\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best hyperparameters for KNeighborsRegressor: {'regressor__n_neighbors': 5, 'regressor__p': 2, 'regressor__weights': 'distance'}
Mean test score: 0.1293
KNeighborsRegressor MSE on test set: 0.0518
KNeighborsRegressor MAE on test set: 0.1754
comment: ได้รับสินค้าแล้วว ส่งก่อนกำหนด4วันเลย สินค้าตรงตามปก ราคาดีไม่แพง เคยใช้แล้วชอบ รู้สึกป้องกันลิปเปื้อนแมสได้ดี
actual: 0.8888888888888888
predicted: 0.9337092804517839

comment: ก่อนซื้อก็อุตส่าห์ดูในรีวิวแล้วนะ ก็เห็นว่าเป็นปุ่มไทยแท้ แต่พอได้รับสินค้ามาก็รู้สึกผิดหวังมาก เป็นสติ๊กเกอร์ไทยซะงั้น ซึ่งพอติดสติ๊กเกอร์แล้ว มันมองไม่เห็นตัวอักษรเลย เวลาใช้งานกลางคืน ลำบากและหงุดหงิดมาก ซื้อแบบมีไฟ แต่ก็เหมือนไม่มีไฟ ช่างเป็นไฟที่ไร้ค่าเสียจริง ผิดหวังจริงๆ รู้งี้ ซื้อร้านอื่น ที่เป็นแป้นไทยแท้ดีกว่า
actual: 0.8888888888888888
predicted: 0.9096256025047901

comment: คุณภาพของสินค้าดีมาก ความคุ้มค่าดีมาก ความเร็วในการจัดส่งดีมากๆ การให้บริการจากร้านค้าดีมาก ถูกใจมากๆ ทุกอย่าง✨
actual: 0.8888888888888888
predicted: 0.8907815358435535

comment: ส่ง